In [15]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/

In [74]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

## Part 1

Goal: given a 
- FIDE ID (corresponding to a player)
- date1
- date2
- threshold **(PENDING)**

output
- Number of tournaments played with end date between date1 and date2 (inclusive)
- Number of games played during those tournaments
- Number of wins, losses, draws as black
- Number of wins, losses, draws as white
- **PENDING**: Number of wins, losses, draws against higher rated player (defined as having a rating of >= player rating + threshold)




# Part 2 **(PENDING)**

Goal: given 
- two FIDE IDs (representing 2 players: player 1 and player 2)
- date1
- date2

output 
- How many times they played each other in a tournament in which the end date is between date1 and date2 (inclusive)
- How many times player 1 wins, draws or loses against player 2

In [83]:
def get_chess_result_data(fide_id_input, date1_input, date2_input):
  ### ex: fide_id_input = 25829092; date1_input = 20160101; date2_input = 20220101
  
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get("https://chess-results.com/SpielerSuche.aspx?lan=1")

  fide_id = wd.find_element("name", "ctl00$P1$txt_fideID")
  fide_id.click()
  fide_id.send_keys(fide_id_input)

  date1 = wd.find_element("name", "ctl00$P1$txt_von_tag")
  date1.click()
  date1.send_keys(date1_input)

  date2 = wd.find_element("name", "ctl00$P1$txt_bis_tag")
  date2.click()
  date2.send_keys(date2_input)

  button = wd.find_element("name", "ctl00$P1$cb_suchen")
  button.click()

  html_from_page = wd.page_source
  soup = BeautifulSoup(html_from_page)

  tournament_links = [x['href'] for x in soup.find_all('a', href=True)]
  tournament_links = [link for link in tournament_links if link.startswith('tnr') and '&snr=' in link]  

  n_tournaments = 0
  n_games = 0
  n_wins_white = 0
  n_losses_white = 0
  n_draws_white = 0
  n_wins_black = 0
  n_losses_black = 0
  n_draws_black = 0

  for link in tqdm(tournament_links):
    cur_link = "https://chess-results.com/" + link
    try:
      html_page = requests.get(cur_link)
      soup = BeautifulSoup(html_page.text)
      black = soup.find_all("div", {"class":"FarbesT"})
      white = soup.find_all("div", {"class":"FarbewT"})
      if len(black) + len(white) > 0:
        n_tournaments += 1
        n_games += len(white) + len(black)
        white_result = [x.findNext('td').text for x in white]
        black_result = [x.findNext('td').text for x in black]
        white_win = white_result.count('1')
        white_loss = white_result.count('0')
        white_draw = len(white) - white_win - white_loss
        black_win = black_result.count('1')
        black_loss = black_result.count('0')
        black_draw = len(black) - black_win - black_loss
        n_wins_white += white_win
        n_losses_white += white_loss
        n_draws_white += white_draw
        n_wins_black += black_win
        n_losses_black += black_loss
        n_draws_black += black_draw
    except:
      continue
  
  return (n_tournaments, n_games, n_wins_white, n_losses_white, 
          n_draws_white, n_wins_black, n_losses_black, n_draws_black)

In [84]:
get_chess_result_data(2093596, 20180101, 20220101)

100%|██████████| 41/41 [00:27<00:00,  1.49it/s]


(40, 380, 120, 25, 43, 95, 47, 50)